# Week 5: Seminar exercises  
## Seminar outline:  

The Transmission Control Protocol (TCP) is a connection-oriented and reliable transport protocol. Unlike UDP, TCP ensures the following:  
- **Reliable data transmission**:  The data is received in order and without  any errors.  
- **Connection management**: It establishes a connection before communication  even begins.  
- **Flow control**: This prevents overwhelming a receiver with too much data.
- **Error recovery**: It retransmits lost or corrupted data/packets.  

| **Feature**       | **TCP** (Reliable) | **UDP** (Fast, Unreliable) |
|------------------|------------------|------------------|
| Connection       | Connection-oriented | Connectionless |
| Reliability      | Ensures ordered & complete delivery | No guarantees of order or delivery |
| Use Case        | Web browsing, emails, file transfer | Streaming, gaming, VoIP |

This seminar  will cover **TCP client-server applications**, **multi-client handling**, and **file transfer** using Python.

Useful information for the exercise below:  
-socket.SOCK_STREAM specifies  the use of TCP.  
-listen(1) enables the server to accept connections (1 queued connection).  
-accept() blocks until a client connects, returning a new socket for that client.  
-sendall() ensures all data is sent (unlike send()).  

In [ ]:
# Task 1: Bulding a simple TCP server
# Objective: Create a TCP server to accept client connections and echo received messages 

import socket
from threading import Thread

def run_server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  
    server_socket.bind(('localhost', 65432))
    server_socket.listen(1)
    print("TCP Server is listening...")

    while True:
        client_socket, client_address = server_socket.accept()
        print(f"Connected to {client_address}")
        data = client_socket.recv(1024)
        print(f"Received: {data.decode()}")
        client_socket.sendall(b"ACK: " + data)
        client_socket.close()

server_thread = Thread(target=run_server)
server_thread.start()

In [ ]:
# Task 2: Building a Simple TCP Client
# Objective: This script allows a TCP client to connect to the server and send messages

import socket
import datetime

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))  

message = input("Enter message: ")
start_time = datetime.datetime.now()
client_socket.sendall(message.encode())

response = client_socket.recv(1024)
end_time = datetime.datetime.now()
time_taken = end_time - start_time

print(f"Server response: {response.decode()}")
print(f"Time taken: {time_taken}")
client_socket.close()

In [ ]:
#Task 3: Logging data in txt file over TCP
# Objective: Modify  the server script to log received data into a text file for logging .


import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind(('localhost', 65432))
server_socket.listen(1)

print("TCP Server is listening and logging messages...")

while True:
    client_socket, client_address = server_socket.accept()
    print(f"Connected to {client_address}")
    
    with open('received_messages.txt', 'a') as f:
        while True:
            data = client_socket.recv(1024)
            if not data:
                break
            f.write(data.decode() + "\n")
            print(f"Logged message: {data.decode()}")
    client_socket.close()

In [ ]:
# Task 4: File transfer over TCP
# Objective: Transfer a file from client to server

# Server (Receives the File - TCP)
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind(('localhost', 65432))
server_socket.listen(1)

print("TCP File Server is listening...")

client_socket, client_address = server_socket.accept()
print(f"Connected to {client_address}")

with open('received_file.txt', 'wb') as f:
    while True:
        data = client_socket.recv(1024)
        if not data:
            break
        f.write(data)
print("File received!")
client_socket.close()

In [ ]:
#Task 5: Multi-Client TCP Chat System (Threading)
#Multi-Client TCP Server
# This script allows multiple clients to connect and chat with each other through a central server.

import socket
import threading

clients = []
clients_lock = threading.Lock()

def handle_client(client_socket, client_address):
    print(f"New connection from {client_address}")
    clients.append(client_socket)
    
    while True:
        try:
            message = client_socket.recv(1024)
            if not message:
                break
            print(f"Message from {client_address}: {message.decode()}")
            broadcast(message, client_socket)
        except ConnectionResetError:
            break
    
    print(f"{client_address} has disconnected")
    clients.remove(client_socket)
    client_socket.close()

def broadcast(message, sender_socket):
    for client in clients:
        if client != sender_socket:
            client.send(message)

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind(('localhost', 65432))
server_socket.listen(5)
print("Multi-Client TCP Server is listening...")

while True:
    client_socket, client_address = server_socket.accept()
    thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
    thread.start()

In [ ]:
#Multi Client chat system
import socket
import threading

def receive_messages(client_socket):
    while True:
        try:
            message = client_socket.recv(1024)
            print(f"\n{message.decode()}\n> ", end="")
        except:
            print("Disconnected from server")
            client_socket.close()
            break

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
client_socket.connect(('localhost', 65432))

thread = threading.Thread(target=receive_messages, args=(client_socket,))
thread.start()

while True:
    message = input("Enter message: ")
    client_socket.send(message.encode())

In [ ]:
# 6. Encrypting Messages for Secure Chat
# Enhancing the chat system with message encryption using the `cryptography` library.

# Encrypted Server
from cryptography.fernet import Fernet
import socket
import threading

# Generate encryption key
key = Fernet.generate_key()
cipher = Fernet(key)
clients = []
clients_lock = threading.Lock()

def handle_client(client_socket, client_address):
    print(f"New connection from {client_address}")
    clients.append(client_socket)
    client_socket.send(key)
    
    while True:
        try:
            encrypted_message = client_socket.recv(1024)
            if not encrypted_message:
                break
            message = cipher.decrypt(encrypted_message).decode()
            print(f"Decrypted Message from {client_address}: {message}")
            broadcast(encrypted_message, client_socket)
        except ConnectionResetError:
            break
    
    print(f"{client_address} has disconnected")
    clients.remove(client_socket)
    client_socket.close()

def broadcast(message, sender_socket):
    for client in clients:
        if client != sender_socket:
            client.send(message)

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind(('localhost', 65432))
server_socket.listen(5)
print("Encrypted Multi-Client TCP Server is listening...")

while True:
    client_socket, client_address = server_socket.accept()
    thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
    thread.start()

In [ ]:
#Encrypted client
from cryptography.fernet import Fernet
import socket
import threading

key = client_socket.recv(1024)
cipher = Fernet(key)

def receive_messages(client_socket):
    while True:
        try:
            encrypted_message = client_socket.recv(1024)
            message = cipher.decrypt(encrypted_message).decode()
            print(f"\n{message}\n> ", end="")
        except:
            print("Disconnected from server")
            client_socket.close()
            break

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
client_socket.connect(('localhost', 65432))

thread = threading.Thread(target=receive_messages, args=(client_socket,))
thread.start()

while True:
    message = input("Enter message: ")
    encrypted_message = cipher.encrypt(message.encode())
    client_socket.send(encrypted_message)

In [ ]:
# Task 7 Fetching Weather Data & Sending Over TCP

import socket
import requests

api_url = "https://api.open-meteo.com/v1/forecast?latitude=51.47&longitude=-0.0363&current_weather=true"
response = requests.get(api_url)

if response.status_code == 200:
    weather_data = response.json()
    temperature = weather_data["current_weather"]["temperature"]
    message = f"Current temperature: {temperature}°C"
else:
    message = "Failed to fetch weather data"

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))
client_socket.sendall(message.encode())
print("Weather data sent!")
client_socket.close()